In [64]:
import numpy as np
import scipy
from scipy.stats import multivariate_normal
import time
from multiprocessing import Pool
import os
import torch
from itertools import repeat


### 2D data

In [65]:
n_components = 30
X = np.random.rand(100000,2)
mus = np.random.rand(n_components,2)
covs = np.array([[1,0],[0,0.5]])
covs = np.tile(covs,(n_components,1,1)).astype(np.float64)
def fun(X_mu_cov):
    X,mu,cov = X_mu_cov
    distribution = multivariate_normal(mean=mu, cov=cov,allow_singular=True)
    distribution.pdf(X)
def fun2(X,mu,cov):
    distribution = multi_pdf(X,mu, cov)

num_workers = os.cpu_count()
X_tensor = torch.from_numpy(X)
mus_tensor = torch.from_numpy(mus)
covs_tensor = torch.from_numpy(covs)

X_batch = torch.from_numpy(X).view(1,-1,2).cuda()
mus_batch = torch.from_numpy(mus).view(n_components,1,2).cuda()
covs_batch = torch.from_numpy(covs).view(n_components,2,2).cuda()

def multi_pdf(X,mu,cov,k=2):
    return torch.exp(-0.5* ((X-mu)**2 @ torch.linalg.inv(cov)).sum(1))/torch.sqrt((2*np.pi)**k*torch.linalg.det(cov))

def batch_multi_pdf(X,mu,cov,k=2):
    return torch.exp(-0.5* ((X-mu)**2 @ torch.linalg.inv(cov)).sum(1))/torch.sqrt((2*np.pi)**k*torch.linalg.det(cov))

    
cpu_normal_time = [];cpu_parallel_time=[];torch_normal_time=[];torch_cuda_time=[];torch_cuda_batch_time = []

for loop in range(100):
    # CPU normal
    start_time = time.time()
    for mu, cov in zip(mus,covs):
        distribution = multivariate_normal(mean=mu, cov=cov,allow_singular=True)
        p1=distribution.pdf(X)
    cpu_normal_time.append(time.time() - start_time)

#     # CPU parallel
#     start_time = time.time()
#     with Pool(num_workers) as p:
#         p.map(fun,zip(X,mus,covs))
#     cpu_parallel_time.append(time.time() - start_time)


#     # torch - CPUtensor([7.0437e-108, 1.0520e-209], device='cuda:0', dtype=torch.float64)
#     start_time = time.time()
#     for mu, cov in zip(mus_tensor,covs_tensor):
#         p2=multi_pdf(X_tensor,mu,cov)
#     torch_normal_time.append(time.time() - start_time)

#     # torch - GPU
    start_time = time.time()
    for mu, cov in zip(mus_tensor,covs_tensor):
        p2=multi_pdf(X_tensor,mu,cov)
    torch_cuda_time.append(time.time() - start_time)
    
    # torch -GPU Batch
    start_time = time.time()
    p3 = torch.exp(-0.5*((X_batch-mus_batch)**2 @ torch.linalg.inv(covs_batch)).sum(2))/torch.sqrt((2*np.pi)**2*torch.linalg.det(covs_batch)).view(-1,1)
    torch_cuda_batch_time.append(time.time() - start_time)
    
    
print(f"cpu_normal_time = \t{np.mean(cpu_normal_time):.4f}"+u" \u00B1 "+f"{np.std(cpu_normal_time):.4f} s")
print(f"cpu_parallel_time = \t{np.mean(cpu_parallel_time):.4f}"+u" \u00B1 "+f"{np.std(cpu_parallel_time):.4f} s")
print(f"torch_normal_time = \t{np.mean(torch_normal_time):.4f}"+u" \u00B1 "+f"{np.std(torch_normal_time):.4f} s")
print(f"torch_cuda_time = \t{np.mean(torch_cuda_time):.4f}"+u" \u00B1 "+f"{np.std(torch_cuda_time):.4f} s")
print(f"torch_cuda_time = \t{np.mean(torch_cuda_batch_time):.4f}"+u" \u00B1 "+f"{np.std(torch_cuda_batch_time):.4f} s")

error = np.linalg.norm(p1-p2.cpu().numpy())
error2 = np.linalg.norm(p1-p3.cpu().numpy()[-1,:])
print(f"check error -> {error:.4e}")
print(f"check error2 -> {error2:.4e}")

AssertionError: Torch not compiled with CUDA enabled

###  3D data

In [ ]:
n_components = 30
X = np.random.rand(1000,3)
mus = np.random.rand(n_components,3)
covs = np.array([[1,0,0.1],[0,1,0.2],[0.1,0.2,0.3]])
covs = np.tile(covs,(n_components,1,1)).astype(np.float64)
X_tensor = torch.from_numpy(X)
mus_tensor = torch.from_numpy(mus)
covs_tensor = torch.from_numpy(covs)
# def fun(X_mu_cov):
#     X,mu,cov = X_mu_cov
#     distribution = multivariate_normal(mean=mu, cov=cov,allow_singular=True)
#     distribution.pdf(X)

# num_workers = os.cpu_count()
# X_tensor = torch.from_numpy(X)
# mus_tensor = torch.from_numpy(mus)
# covs_tensor = torch.from_numpy(covs)
# def multi_pdf(X,mu,cov):
#     N,k = X.size()
#     v = X-mu
#     x_temp = -0.5* ((X-mu) @ torch.linalg.inv(cov)*(X-mu)).sum(1) # N-by-3 matrix
#     numerator = torch.exp(x_temp)
#     denomitor = torch.sqrt((2*np.pi)**k*torch.linalg.det(cov))
#     return numerator/denomitor
    
cpu_normal_time = [];cpu_parallel_time=[];torch_normal_time=[];torch_cuda_time=[]

for loop in range(10):
    # CPU normal
    start_time = time.time()
    for mu, cov in zip(mus,covs):
        distribution = multivariate_normal(mean=mu, cov=cov,allow_singular=True)
        p1=distribution.pdf(X)
    cpu_normal_time.append(time.time() - start_time)

    # CPU parallel
    start_time = time.time()
    with Pool(num_workers) as p:
        p.map(fun,zip(X,mus,covs))
    cpu_parallel_time.append(time.time() - start_time)


    # torch - CPUtensor([7.0437e-108, 1.0520e-209], device='cuda:0', dtype=torch.float64)
    start_time = time.time()
    for mu, cov in zip(mus_tensor,covs_tensor):
        p2=multi_pdf(X_tensor,mu,cov,k=3)
    torch_normal_time.append(time.time() - start_time)

    # torch - GPU
    start_time = time.time()
    for mu, cov in zip(mus_tensor,covs_tensor):
        p2=multi_pdf(X_tensor,mu,cov,k=3)
    torch_cuda_time.append(time.time() - start_time)

print(f"cpu_normal_time = \t{np.mean(cpu_normal_time):.4f}"+u" \u00B1 "+f"{np.std(cpu_normal_time):.4f} s")
print(f"cpu_parallel_time = \t{np.mean(cpu_parallel_time):.4f}"+u" \u00B1 "+f"{np.std(cpu_parallel_time):.4f} s")
print(f"torch_normal_time = \t{np.mean(torch_normal_time):.4f}"+u" \u00B1 "+f"{np.std(torch_normal_time):.4f} s")
print(f"torch_cuda_time = \t{np.mean(torch_cuda_time):.4f}"+u" \u00B1 "+f"{np.std(torch_cuda_time):.4f} s")

error = np.linalg.norm(p1-p2.cpu().numpy())
print(f"check error -> {error:.4e}")


In [ ]:
1080*720

In [ ]:
120*80/10